In [ ]:
#| default_exp data.core

# Data core

> Core functionality to transform and gather data

In [ ]:
#|export
from __future__ import annotations
from fastcore.basics import store_attr
import pandas as pd
import re
from pathlib import Path
from tqdm import tqdm